In [1]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import pandas as pd
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction
from SBMLLint.games.games_report import NULL_STR, NUM_STAR, PARAGRAPH_DIVIDER, REPORT_DIVIDER

ReactionOperation = collections.namedtuple("ReactionOperation", 
    "reaction operation")

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [38]:
# Model 248 is for canceling error
# Model 14/167 for a type three error / echelon
# Model 7 for type 1 error
# Model 18 for type 2 error

######## examine model 628 for large echelon error ###########
simple = load_file_from_curated_data(18)
for r in simple.reactions:
  if r.category != cn.REACTION_BOUNDARY and r.category != cn.REACTION_1_1:
    print(r.makeIdentifier(is_include_kinetics=False))
m = GAMES_PP(simple)
m.analyze()
gr = GAMESReport(m)

SHMT: FH4 + serine -> CH2FH4
HCHOtoCH2FH4: FH4 + HCHO -> CH2FH4
CH2FH4toHCHO: CH2FH4 -> FH4 + HCHO
MTHFR: CH2FH4 + NADPH -> CH3FH4
MTR: CH3FH4 + homocysteine -> FH4
HCOOHtoCHOFH4: FH4 + formate + ATP -> CHOFH4
GARFT: CHOFH4 + GAR -> FGAR + FH4
ATIC7: CHOFH4 + AICAR -> FH4
MTHFD: CH2FH4 + NADP -> CHOFH4
TYMS: CH2FH4 + dUMP -> FH2f
ATIC12: FFH2 + AICAR -> FH2f
MTX1on: MTX1 + DHFRf -> MTX1b
MTX2on: MTX2 + DHFRf -> MTX2b
MTX3on: MTX3 + DHFRf -> MTX3b
MTX4on: MTX4 + DHFRf -> MTX4b
MTX5on: MTX5 + DHFRf -> MTX5b
MTX1off: MTX1b -> MTX1 + DHFRf
MTX2off: MTX2b -> MTX2 + DHFRf
MTX3off: MTX3b -> MTX3 + DHFRf
MTX4off: MTX4b -> MTX4 + DHFRf
MTX5off: MTX5b -> MTX5 + DHFRf
We just analyzed the data...
Type I error:  [PathComponents(node1='FH4', node2='CH2FH4', reactions=['CH2FH4toHCHO', 'SHMT', 'HCHOtoCH2FH4']), PathComponents(node1='CH2FH4', node2='FH2f', reactions=['TYMS']), PathComponents(node1='FFH2', node2='FH2f', reactions=['ATIC12']), PathComponents(node1='MTX1', node2='MTX1b', reactions=['MTX1

In [42]:
m.type_two_errors[0]

[{CH3FH4}, {CH2FH4=FFH2=FH2f=FH4}]

In [45]:
type2_report, error_num = gr.reportTypeTwoError([m.type_two_errors[0]], explain_details=True)
error_num

[5]

In [47]:
print(type2_report)

We detected a mass imbalance from the following reactions:

These uni-uni reactions created mass-equivalence.

1. DHFReductase: FH2f -> FH4
2. FFH2syn: FH2f -> FFH2
3. SHMTr: CH2FH4 -> FH4

----------------------------------------------------------------------

The following reactions create mass-inequality.

4. MTR: CH3FH4 + homocysteine -> FH4
5. MTHFR: CH2FH4 + NADPH -> CH3FH4

----------------------------------------------------------------------

Based on the reactions above, we have mass-equivalent pseudo reactions.

(pseudo 4.) MTR: {CH3FH4} + {homocysteine} -> {CH2FH4=FFH2=FH2f=FH4}
(pseudo 5.) MTHFR: {CH2FH4=FFH2=FH2f=FH4} + {NADPH} -> {CH3FH4}

----------------------------------------------------------------------

However, the above pseudo reactions imply the following inequalities:

{CH3FH4} < {CH2FH4=FFH2=FH2f=FH4} < {CH3FH4}

This indicates a mass conflict between reactions.

----------------------------------------------------------------------

-------------------------

In [49]:
k = "This indicates a mass conflict between reactions.\n\n----------------------------------------------------------------------\n\n----------------------------------------------------------------------\n\n"
len(k)

241

In [48]:
type2_report

'We detected a mass imbalance from the following reactions:\n\nThese uni-uni reactions created mass-equivalence.\n\n1. DHFReductase: FH2f -> FH4\n2. FFH2syn: FH2f -> FFH2\n3. SHMTr: CH2FH4 -> FH4\n\n----------------------------------------------------------------------\n\nThe following reactions create mass-inequality.\n\n4. MTR: CH3FH4 + homocysteine -> FH4\n5. MTHFR: CH2FH4 + NADPH -> CH3FH4\n\n----------------------------------------------------------------------\n\nBased on the reactions above, we have mass-equivalent pseudo reactions.\n\n(pseudo 4.) MTR: {CH3FH4} + {homocysteine} -> {CH2FH4=FFH2=FH2f=FH4}\n(pseudo 5.) MTHFR: {CH2FH4=FFH2=FH2f=FH4} + {NADPH} -> {CH3FH4}\n\n----------------------------------------------------------------------\n\nHowever, the above pseudo reactions imply the following inequalities:\n\n{CH3FH4} < {CH2FH4=FFH2=FH2f=FH4} < {CH3FH4}\n\nThis indicates a mass conflict between reactions.\n\n------------------------------------------------------------------

In [15]:
G2K = "G2K"
PG2R = "PG2R"
som = m.getNode(simple.getMolecule("G2K"))
gr.getMoleculeEqualityPath(som, "G2K", "PG2R")

[PathComponents(node1='G2K', node2='PG2', reactions=['Cdc2Phos']),
 PathComponents(node1='PG2', node2='PG2R', reactions=['Rum1DegInPG2R'])]

In [21]:
count, report = gr.getMoleculeEqualityPathReport(G2K, PG2R, 0, explain_details=True)
print(count)
report

2


'\nG2K = PG2 by reaction(s):\n1. Cdc2Phos: G2K -> PG2\n\nPG2 = PG2R by reaction(s):\n2. Rum1DegInPG2R: PG2R -> PG2\n'

In [28]:
count, report = gr.getMoleculeInequalityPathReport(G2K, PG2R, ["G2R_Creation"], 0, explain_details=True)
print(count)
report

1


'G2K < PG2R by reaction(s):\n1. G2R_Creation: G2K + R -> G2R\n'

In [31]:
m.type_one_errors[0]

PathComponents(node1='G2K', node2='G2R', reactions=['G2R_Creation'])

In [32]:
cn.PathComponents(node1='a', node2='b', reactions=['1'])

PathComponents(node1='a', node2='b', reactions=['1'])

In [34]:
# From model 7
type1_report, error_num = gr.reportTypeOneError([m.type_one_errors[0]], explain_details=True)
print(error_num)
type1_report

[2]


'We detected a mass imbalance from the following reactions:\n\n\nG2K = G2R by reaction(s):\n1. Rum1DegInG2R: G2R -> G2K\n\nHowever, G2K < G2R by reaction(s):\n2. G2R_Creation: G2K + R -> G2R\n\n\n----------------------------------------------------------------------\n\n\n\n**********************************************************************\n\n'

In [5]:
# From model 248
canceling_report, error_num = gr.reportCancelingError(m.canceling_errors, explain_details=True)
canceling_report

'We detected a mass imbalance from the following reactions:\n\n1. OxidativePhosphorylation: 6.00 ADP + CTtis -> 6.00 ATP\n\n2. ATPase: ATP -> ADP\n\n*ATP and ADP have the same mass according to the above reaction\n\n\nTherefore, they will result in empty product with zero mass:\n\nOxidativePhosphorylation: CTtis -> \n\nThis indicates a mass conflict between reactions.\n\n----------------------------------------------------------------------\n\n----------------------------------------------------------------------\n\n\n\n**********************************************************************\n\n'

In [6]:
correct_report = NULL_STR
correct_report = correct_report + "We detected a mass imbalance from the following reactions:\n\n"
correct_report = correct_report + "1. OxidativePhosphorylation: 6.00 ADP + CTtis -> 6.00 ATP\n\n"
correct_report = correct_report + "2. ATPase: ATP -> ADP\n\n"
correct_report = correct_report + "*ATP and ADP have the same mass according to the above reaction\n\n\n"
correct_report = correct_report + "Therefore, they will result in empty product with zero mass:\n\n"
correct_report = correct_report + "OxidativePhosphorylation: CTtis -> \n\n"
correct_report = correct_report + "This indicates a mass conflict between reactions."
correct_report = correct_report + "\n%s%s\n" % (PARAGRAPH_DIVIDER, PARAGRAPH_DIVIDER)
correct_report + correct_report + "\n%s\n" % REPORT_DIVIDER

correct_report

'We detected a mass imbalance from the following reactions:\n\n1. OxidativePhosphorylation: 6.00 ADP + CTtis -> 6.00 ATP\n\n2. ATPase: ATP -> ADP\n\n*ATP and ADP have the same mass according to the above reaction\n\n\nTherefore, they will result in empty product with zero mass:\n\nOxidativePhosphorylation: CTtis -> \n\nThis indicates a mass conflict between reactions.\n\n----------------------------------------------------------------------\n\n----------------------------------------------------------------------\n\n'

In [7]:
simple.reactions

[ATPase: ATP -> ADP; Tissue * Katpase * ATP,
 OxidativePhosphorylation: 6.00 ADP + CTtis -> 6.00 ATP; Tissue * Vmax * (CFtis / (Km + CFtis)) * ADP / (Kadp + ADP),
 ConvectionTransport:  -> CTcap; Qm * (CTart - CTcap),
 DiffusionTransport: CTcap -> CTtis; PSm * (CFcap - CFtis),
 CreatineKinase: ADP + PCr -> ATP + Cr; Tissue * (VfCK * ADP * PCr / (Kb * Kia) - VrCK * Cr * ATP / (Kiq * Kp)) / ((Kia + ADP) / Kia + ATP / Kiq + PCr / Kib + ADP * PCr / (Kb * Kia) + Cr * ATP / (Kiq * Kp))]

In [8]:
m.nodes

NodeView(({PCr}, {Cr}, {ADP=ATP}, {CTcap=CTtis}))